In [1]:
import sys
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extrair_inteiro(texto):
	try:
		i = texto.rindex(' ')
		sem_unidade = texto[:i]

		# Às vezes, esse valor pode iniciar pelo ano...
		i = sem_unidade.find(' ')
		if i >= 0:
			sem_unidade = sem_unidade[(i + 1):]

		sem_virgula = sem_unidade.replace(',', '')

		return int(sem_virgula)
	except:
		return 0

driver = webdriver.Chrome()
driver.get('https://www.imdb.com/pt/chart/moviemeter/?ref_=shlc_nv_menu')

The chromedriver version (141.0.7390.78) detected in PATH at C:\SeleniumDrivers\chromedriver.exe might not be compatible with the detected chrome version (142.0.7444.134); currently, chromedriver 142.0.7444.162 is recommended for chrome 142.*, so it is advised to delete the driver in PATH and retry


In [2]:
nome_filme= WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h3.ipc-title__text.ipc-title__text--reduced'))
)
print(nome_filme[2].text.strip())


Michael


In [6]:
data_filme= WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.sc-caa65599-7.eeMIpC.cli-title-metadata-item'))
)
print(data_filme[0].text.strip())


2025


In [63]:
tempo_filme= WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.sc-caa65599-7.eeMIpC.cli-title-metadata-item'))
)
print(tempo_filme[4].text.strip())


2h 8m


In [64]:
idade_filme= WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.sc-caa65599-7.eeMIpC.cli-title-metadata-item'))
)
print(idade_filme[5].text.strip())

18


In [65]:
nota_filme= WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.ipc-rating-star--rating'))
)
print(nota_filme[1].text.strip())

7,5


In [66]:
botao_filme= WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.ipc-lockup-overlay__screen'))
)
botao_filme[1].click()

In [67]:
personagems_filme= WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a.sc-10bde568-1.jBmamV')))

In [68]:
atores_filme= WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'span.sc-10bde568-4.jwxYun'))
)

In [69]:
dados = []

for i in range(len(personagems_filme)):
	personagems = personagems_filme[i].text.strip()
	atores = atores_filme[i].text.strip()

	dados.append({
		"personagem": personagems,
		"atores": atores
	})
print(dados)
driver.quit()
 

[{'personagem': 'Scarlett Sher', 'atores': 'Narrator'}, {'personagem': 'Julia Garner', 'atores': 'Justine'}, {'personagem': 'Cary Christopher', 'atores': 'Alex'}, {'personagem': 'Jason Turner', 'atores': 'Counselor'}, {'personagem': 'Josh Brolin', 'atores': 'Archer'}, {'personagem': 'Benedict Wong', 'atores': 'Marcus'}, {'personagem': 'Anny Jules', 'atores': 'Parent #1'}, {'personagem': 'Ali Burch', 'atores': 'Parent #2'}, {'personagem': 'Michael Gene Conti', 'atores': 'Parent #3'}, {'personagem': 'Austin Abrams', 'atores': 'James'}, {'personagem': 'Eric Jepson', 'atores': 'Officer'}, {'personagem': 'Alden Ehrenreich', 'atores': 'Paul'}, {'personagem': 'Whitmer Thomas', 'atores': "Alex's Dad"}, {'personagem': 'Callie Schuttera', 'atores': "Alex's Mom"}, {'personagem': 'June Diane Raphael', 'atores': 'Donna'}, {'personagem': 'Ronny Mathew', 'atores': 'Liquor Store Clerk'}, {'personagem': 'Amy Madigan', 'atores': 'Gladys'}, {'personagem': 'Melissa Ponzio', 'atores': "Archer's Wife"}]
